In [1]:
import requests
from bs4 import BeautifulSoup

def is_target_tag(tag):
    target_tags = ['p', 'code', 'h1', 'h2', 'h2', 'h3', 'h4', 'h5']
    return tag.name in target_tags

def get_text_from_url(url):
    response = requests.get(url)
    html = response.text
    soup = BeautifulSoup(html, 'html.parser')
    extracted_tags = soup.find_all(is_target_tag)
    res_text = ''
    for tag in extracted_tags:
        res_text = res_text + tag.get_text(strip=True) + '\n'
    return res_text

In [13]:
functions = [
    {
        "name": "get_text_from_url",
        "description": "抓取url对应的网页里的文本内容",
        "parameters": {
            "type": "object",
            "properties": {
                "url": {
                    "type": "string",
                    "description": "网址url",
                }
            },
            "required": ["url"],
        },
    }
]

In [4]:
available_functions = {
    "get_text_from_url": get_text_from_url,
}

In [15]:
import os
import openai
import json
openai.api_key = os.getenv("OPENAI_API_KEY")
openai.proxy = "http://127.0.0.1:7890"

def chat(content):
    messages = [
        {"role": "user", "content": content}
    ]
    response1 = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-0613",
        messages=messages,
        functions=functions,
        function_call="auto"
    )
    response_message = response1["choices"][0]["message"]
    rtn_message = response1["choices"][0]["message"]
    # 如果ChatGPT返回结果会告诉你，是否需要调用函数，我们只需要根据它返回的函数名、参数调起对应的函数
    # 然后将函数的返回结果再给到ChatGPT，让他进行下一步的操作
    if response_message.get("function_call"):
        # 找到需要调用的函数，并将ChatGPT给的参数传进去
        function_name = response_message["function_call"]["name"]
        fuction_to_call = available_functions[function_name]
        function_args = json.loads(response_message["function_call"]["arguments"])
        # 用这种方式可以调起任意python函数，不用像官网那样还要指定参数名
        function_response = fuction_to_call(**function_args)
        # 获取到函数调用结果后，需要将结果拼接到对话记录里，并再次调用ChatGPT
        messages.append(response_message)
        messages.append(
            {
                "role": "function",
                "name": function_name,
                "content": function_response,
            }
        )
        # 二次调用的返回结果里就是我们预期的结果了
        response2 = openai.ChatCompletion.create(
            model="gpt-3.5-turbo-0613",
            messages=messages,
        )
        rtn_message = response2["choices"][0]["message"]
    return rtn_message['content']

In [16]:
chat('总结下这篇文章，将其中的要点提炼出来 https://zxs.io/article/1924')

'该文章讨论了如何使用ChatGPT来提升编程能力。作者总结了可以利用ChatGPT的一些场景，包括代码评审、代码优化、代码释义、提供解决方案、代码生成、设计模式和架构建议以及学习新知识。作者强调了在使用ChatGPT时应注意其表现水平可能不如专业工程师，并建议将其作为辅助工具结合其他代码审查工具和人工审查使用。最后，作者提到尽管ChatGPT可能不会完全取代程序员，但肯定会替代部分技术水平较低的工作。'